In [26]:
import pandas as pd
import os

df = pd.read_csv('./datasets/forest_features_20240701.csv')
df.head()

,date,1wk_WIS,1_week_roc,is_wis_above_avg,1st_deriv_stencil,week_of_year,moving_avg_WIS,lagged_1wk_WIS,loc_code,holiday_last_week,population
0,2023-10-28,19.51,0.320000,0,7.333333,43,13.273333,13.51,18,0,6785528
1,2023-11-04,6.51,0.181818,0,4.166667,44,13.176667,19.51,18,0,6785528
2,2023-11-11,45.84,0.205128,0,27.750000,45,23.953333,6.51,18,0,6785528
3,2023-11-18,11.54,0.851064,0,16.416667,46,21.296667,45.84,18,1,6785528
4,2023-11-25,37.36,-0.034483,0,25.750000,47,31.580000,11.54,18,0,6785528


In [27]:
# Add the median quantile prediction and actual hospitalizations
all_data = []

directory = 'LosAlamos_NAU-CModel_Flu'
for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    csv_df = pd.read_csv(file_path)
    
    # Filter for median predictions
    filtered_csv_df = csv_df[csv_df["output_type_id"] == "0.5"]
    filtered_csv_df = filtered_csv_df[filtered_csv_df["horizon"] == 1]
    filtered_csv_df.rename(columns={"reference_date": "date", "location": "loc_code"}, inplace=True)
    
    all_data.append(filtered_csv_df)

aggregated_df = pd.concat(all_data, ignore_index=True)
aggregated_df

,date,target,horizon,target_end_date,loc_code,output_type,output_type_id,value
0,2023-10-21,wk inc flu hosp,1,2023-10-28,11,quantile,0.5,0.000000
1,2023-10-21,wk inc flu hosp,1,2023-10-28,72,quantile,0.5,117.498152
2,2023-10-21,wk inc flu hosp,1,2023-10-28,12,quantile,0.5,198.435375
3,2023-10-21,wk inc flu hosp,1,2023-10-28,01,quantile,0.5,14.159718
4,2023-10-21,wk inc flu hosp,1,2023-10-28,02,quantile,0.5,7.044904
...,...,...,...,...,...,...,...,...
1265,2024-04-06,wk inc flu hosp,1,2024-04-13,53,quantile,0.5,38.237101
1266,2024-04-06,wk inc flu hosp,1,2024-04-13,55,quantile,0.5,138.394057
1267,2024-04-06,wk inc flu hosp,1,2024-04-13,54,quantile,0.5,61.235295
1268,2024-04-06,wk inc flu hosp,1,2024-04-13,56,quantile,0.5,3.975788


In [28]:
df['loc_code'] = df['loc_code'].astype('int32')
df.head()


,date,1wk_WIS,1_week_roc,is_wis_above_avg,1st_deriv_stencil,week_of_year,moving_avg_WIS,lagged_1wk_WIS,loc_code,holiday_last_week,population
0,2023-10-28,19.51,0.320000,0,7.333333,43,13.273333,13.51,18,0,6785528
1,2023-11-04,6.51,0.181818,0,4.166667,44,13.176667,19.51,18,0,6785528
2,2023-11-11,45.84,0.205128,0,27.750000,45,23.953333,6.51,18,0,6785528
3,2023-11-18,11.54,0.851064,0,16.416667,46,21.296667,45.84,18,1,6785528
4,2023-11-25,37.36,-0.034483,0,25.750000,47,31.580000,11.54,18,0,6785528


In [30]:
aggregated_df = aggregated_df[aggregated_df['loc_code'] != 'US']
aggregated_df['loc_code'] = aggregated_df['loc_code'].astype('int32')
aggregated_df.head()

/tmp/ipykernel_803262/4094823712.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_df['loc_code'] = aggregated_df['loc_code'].astype('int32')


,date,target,horizon,target_end_date,loc_code,output_type,output_type_id,value
0,2023-10-21,wk inc flu hosp,1,2023-10-28,11,quantile,0.5,0.000000
1,2023-10-21,wk inc flu hosp,1,2023-10-28,72,quantile,0.5,117.498152
2,2023-10-21,wk inc flu hosp,1,2023-10-28,12,quantile,0.5,198.435375
3,2023-10-21,wk inc flu hosp,1,2023-10-28,1,quantile,0.5,14.159718
4,2023-10-21,wk inc flu hosp,1,2023-10-28,2,quantile,0.5,7.044904


In [31]:
merged_df = pd.merge(df, aggregated_df[["date", "loc_code", "value"]], how="left", left_on=["date", "loc_code"], right_on=["date", "loc_code"])
merged_df

,date,1wk_WIS,1_week_roc,is_wis_above_avg,1st_deriv_stencil,week_of_year,moving_avg_WIS,lagged_1wk_WIS,loc_code,holiday_last_week,population,value
0,2023-10-28,19.51,0.320000,0,7.333333,43,13.273333,13.51,18,0,6785528,14.324939
1,2023-11-04,6.51,0.181818,0,4.166667,44,13.176667,19.51,18,0,6785528,14.324939
2,2023-11-11,45.84,0.205128,0,27.750000,45,23.953333,6.51,18,0,6785528,38.088067
3,2023-11-18,11.54,0.851064,0,16.416667,46,21.296667,45.84,18,1,6785528,38.088067
4,2023-11-25,37.36,-0.034483,0,25.750000,47,31.580000,11.54,18,0,6785528,84.159595
...,...,...,...,...,...,...,...,...,...,...,...,...
1113,2024-03-02,7.56,-0.173077,0,-1.000000,9,4.150000,1.44,72,1,3285874,41.556136
1114,2024-03-09,13.41,0.209302,0,14.166667,10,7.470000,7.56,72,0,3285874,41.556136
1115,2024-03-16,2.61,0.230769,0,-1.750000,11,7.860000,13.41,72,0,3285874,42.345551
1116,2024-03-23,2.18,-0.218750,0,-8.000000,12,6.066667,2.61,72,0,3285874,44.852743
